In [1]:
import os
from lxml import etree

In [2]:
# Load the XML file
xml_file = 'example/dspace-oai-dc.xml'
xml_tree = etree.parse(xml_file)

# Load the XSLT file
xslt_file = 'xsl/DC2MARC21slim.xsl'
xslt_tree = etree.parse(xslt_file)

# Perform the transformation
transform = etree.XSLT(xslt_tree)
result_tree = transform(xml_tree)

# Create the output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Generate the output filename
base_name = os.path.basename(xml_file)
name, ext = os.path.splitext(base_name)
output_file = os.path.join(output_dir, f'{name}_transformed{ext}')

# Save the result to the new file
result_tree.write(output_file, pretty_print=True, xml_declaration=True, encoding='UTF-8')

print(f'Transformation complete. Output saved to {output_file}')

Transformation complete. Output saved to output/dspace-oai-dc_transformed.xml


In [5]:

# Load the XML file
xml_file = 'example/archivesspace-ead.xml'
xml_tree = etree.parse(xml_file)

# Load the XSLT file
xslt_file = 'xsl/EAD2MARC21_own.xsl'
xslt_tree = etree.parse(xslt_file)

# Perform the transformation
transform = etree.XSLT(xslt_tree)
result_tree = transform(xml_tree)

# Create the output directory if it doesn't exist
output_dir = 'output'
os.makedirs(output_dir, exist_ok=True)

# Generate the output filename
base_name = os.path.basename(xml_file)
name, ext = os.path.splitext(base_name)
output_file = os.path.join(output_dir, f'{name}_transformed{ext}')

# Save the result to the new file
result_tree.write(output_file, pretty_print=True, xml_declaration=True, encoding='UTF-8')

print(f'Transformation complete. Output saved to {output_file}')

Transformation complete. Output saved to output/archivesspace-ead_transformed.xml
